In [19]:
import re
import glob
import csv
from helpers import config
from helpers.loading import load_daily_data ,file_exist,get_all_dates
from helpers.algorithm import find_best_delay
import pandas as pd
import time
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
all_dates = get_all_dates(config["stock"])
print(f"{len(all_dates)} dates to process")

3034 dates to process


In [ ]:
fieldnames = ['date', 'market1','market2',"lag"]
preprocessing_steps = ['numeric',"log_returns"]

# file where to write the computed lags
results_path = config["files"]["results"]["all_best_lags"].format("_".join(preprocessing_steps))

result_file_exists = file_exist(results_path) 
csvfile = open(results_path, 'a', newline='') 
writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

if result_file_exists:
    processed_dates = set(pd.read_csv(results_path).date.unique())
else: 
    # if the file is new, we need to write headers
    writer.writeheader()
    processed_dates = set()


max_iterations = 5000
start_time = time.time()
date_count = 0 # number of dates processed
for date_id,date in enumerate(all_dates[::1]):
    print(f"date:{date}, {date_id}:{len(all_dates)}, {100*date_id/len(all_dates):0.3f}%", end="\r")

    daily_data = load_daily_data(date,preprocessing_steps=preprocessing_steps)
    if not daily_data:
        # in case all markets do not provide data for the given date, we skip the date
        continue
    
    # we skip the current date if it has already been processed
    if date in processed_dates:
        continue
        
    for i,n1 in enumerate(daily_data):
        for j,n2 in enumerate(daily_data):
            if i>j: # avoid symetric (corr(a,b)=corr(b,a)) and meaningless (corr(a,a)=1) calculations
                best_delay, delays, correlations, los, his = find_best_delay(daily_data,n1,n2,step_size=1000)
                # write the computed result
                writer.writerow({'date': date, 'market1': n1,'market2': n2,'lag': best_delay})
                writer.writerow({'date': date, 'market1': n2,'market2': n1,'lag': -best_delay})
    csvfile.flush() # flush every time we processed a date
    date_count+=1
    if date_count>=max_iterations:
        break
print()
print(f"{date_count} dates processed in {time.time()-start_time:0.2f}s")
csvfile.close()

In [22]:
daily_data

{'GB':                                         price
 date                                         
 2010-07-12 07:00:12.564000+00:00     0.000000
 2010-07-12 07:00:13.581000192+00:00  0.000000
 2010-07-12 07:00:13.599000064+00:00 -0.001128
 2010-07-12 07:00:13.606000640+00:00  0.000000
 2010-07-12 07:00:13.839000064+00:00  0.000000
 ...                                       ...
 2010-07-12 15:29:48.189999616+00:00  0.000000
 2010-07-12 15:29:48.240000+00:00     0.000000
 2010-07-12 15:29:48.304999680+00:00  0.000000
 2010-07-12 15:29:48.309999872+00:00  0.000000
 2010-07-12 15:29:49.120999936+00:00  0.000000
 
 [1839 rows x 1 columns],
 'US':                                         price
 date                                         
 2010-07-12 13:32:38.300000+00:00     0.000377
 2010-07-12 13:32:45.878999808+00:00  0.000377
 2010-07-12 13:32:49.690999552+00:00  0.000000
 2010-07-12 13:32:57.945000192+00:00  0.000000
 2010-07-12 13:33:03.484999680+00:00  0.000000
 ...                